In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

def load_custom_data(session_folder_path):
    """
    Loads the combined data and metadata from a custom recording session.

    Args:
        session_folder_path (str): Path to the session folder.

    Returns:
        tuple: (data_array, metadata)
               - data_array (np.ndarray): The loaded data, transposed to (samples, channels).
               - metadata (dict): The loaded session_info from metadata.
               Returns (None, None) if loading fails.
    """
    data_filename = "custom_combined_data.dat"
    metadata_filename = "custom_metadata.npz"

    data_filepath = os.path.join(session_folder_path, data_filename)
    metadata_filepath = os.path.join(session_folder_path, metadata_filename)

    if not os.path.exists(metadata_filepath):
        print(f"Error: Metadata file not found: {metadata_filepath}")
        return None, None
    
    session_info_loaded = None
    try:
        metadata_loaded = np.load(metadata_filepath, allow_pickle=True)
        session_info_loaded = metadata_loaded['session_info'].item()
    except Exception as e_meta:
        print(f"Error loading metadata from {metadata_filepath}: {e_meta}")
        return None, None # Cannot proceed without metadata

    if not os.path.exists(data_filepath):
        print(f"Error: Data file not found: {data_filepath}")
        # Return metadata if it was loaded, so user can inspect session_info
        return None, session_info_loaded 

    try:
        num_channels = session_info_loaded['expected_columns'] # Should be 8
        data_type = np.dtype(session_info_loaded['custom_data_type'])
        data_shape_on_save = session_info_loaded.get('data_shape_on_save', 'samples_first') 

        loaded_flat_data = np.fromfile(data_filepath, dtype=data_type)

        if loaded_flat_data.size == 0:
            print("Warning: Data file is empty.")
            return np.array([]).reshape(0, num_channels), session_info_loaded 

        if data_shape_on_save == 'channels_first':
            if loaded_flat_data.size % num_channels == 0:
                num_samples_loaded_total = loaded_flat_data.size // num_channels
                # Reshape as (channels, total_samples_in_file)
                reshaped_data_channels_first = loaded_flat_data.reshape(num_channels, num_samples_loaded_total)
                # Transpose to (total_samples_in_file, channels)
                final_data_samples_first = reshaped_data_channels_first.T 
                return final_data_samples_first, session_info_loaded
            else:
                print(f"Error: Cannot reshape data saved as 'channels_first'. Total elements ({loaded_flat_data.size}) "
                      f"not divisible by num_channels ({num_channels}).")
                return None, session_info_loaded
        else: # Assuming 'samples_first' or old format where num_channels was num_cols
            if loaded_flat_data.size % num_channels == 0: 
                num_samples_loaded = loaded_flat_data.size // num_channels
                reshaped_data = loaded_flat_data.reshape(num_samples_loaded, num_channels)
                return reshaped_data, session_info_loaded
            else:
                print(f"Error: Cannot reshape data saved as 'samples_first'. Total elements ({loaded_flat_data.size}) "
                      f"not divisible by num_columns ({num_channels}).")
                return None, session_info_loaded

    except Exception as e:
        print(f"Error processing data from {data_filepath} or applying metadata: {e}")
        import traceback
        traceback.print_exc()
        # Return metadata if it was loaded, so user can inspect session_info
        return None, session_info_loaded

# --- Configuration for loading ---
# !!! IMPORTANT !!!
# Replace 'YOUR_SESSION_FOLDER_HERE' with the actual name of the 
# subfolder in 'recorded_data/' that was created by the modified 'test_custom_save.py'.
# For example: SESSION_FOLDER_PATH = "recorded_data/20250507_103045_123456"


# <<< PLEASE UPDATE THIS PATH >>>
SESSION_FOLDER_PATH = "recorded_data/ALTERNESS_4minmark_2"

print(f"Attempting to load data from: {SESSION_FOLDER_PATH}")
loaded_data, session_metadata = load_custom_data(SESSION_FOLDER_PATH)

if session_metadata is not None: 
    print("\nSession Information (from metadata):")
    for key, value in session_metadata.items():
        print(f"  {key}: {value}")

    if loaded_data is not None:
        print("\nSuccessfully loaded data.")
        print(f"Data shape (samples, channels): {loaded_data.shape}")
        
        if loaded_data.shape[0] > 0: 
            print("\nFirst 5 rows of loaded data (transposed to samples, channels):")
            column_names = session_metadata.get('column_names', [f'Ch{i+1}' for i in range(loaded_data.shape[1])])
            header = " | ".join(column_names)
            print(header)
            print("-" * len(header))
            for row in loaded_data[:5, :]:
                print(" | ".join(map(lambda x: f"{x:.3f}" if not np.isnan(x) else "NaN", row)))
        else:
            print("\nData loaded, but no samples to display (data shape is 0 rows).")

        # --- Optional: Example of plotting the first EEG channel ---
        # if 'column_names' in session_metadata and loaded_data.shape[0] > 0 and loaded_data.shape[1] > 0:
        #     first_eeg_channel_name = 'EEG_Filt_1' 
        #     if first_eeg_channel_name in session_metadata['column_names']:
        #         try:
        #             eeg_channel_index = session_metadata['column_names'].index(first_eeg_channel_name)
        #             plt.figure(figsize=(15, 5))
        #             plt.plot(loaded_data[:, eeg_channel_index]) 
        #             plt.title(f"Plot of: {session_metadata['column_names'][eeg_channel_index]}")
        #             plt.xlabel("Sample Index")
        #             plt.ylabel("Value")
        #             plt.grid(True)
        #             plt.show()
        #         except IndexError:
        #              print(f"\nSkipping plot: Channel index for '{first_eeg_channel_name}' out of bounds for loaded data shape {loaded_data.shape}.")
        #     else:
        #         print(f"\nSkipping plot: Channel '{first_eeg_channel_name}' not found in column names: {session_metadata['column_names']}.")
        # else:
        #     print("\nSkipping plot: Conditions not met (column names missing, no samples, or no channels).")

    else: 
        print(f"\nFailed to load data array from {SESSION_FOLDER_PATH}, but metadata was available.")
        print("Please check data file integrity and error messages above.")
else: 
    print(f"\nFailed to load any data or metadata from {SESSION_FOLDER_PATH}.")
    print("Please check the following:")
    print("1. The 'test_custom_save.py' script has been run successfully to generate data.")
    print("2. The 'SESSION_FOLDER_PATH' variable in this cell is correctly set to the generated session folder.")
    print("   (e.g., 'recorded_data/YYYYMMDD_HHMMSS_micros')")
    print("3. The session folder contains 'custom_combined_data.dat' and 'custom_metadata.npz'.")

Attempting to load data from: recorded_data/ALTERNESS_4minmark_2

Session Information (from metadata):
  product_key: RUtYA4W3kpXi0i9C7VZCQJY5_GRhm4XL2rKp6cviwQI=
  device_id: FRENZI40
  session_start_iso: 2025-05-07T23:42:34.416379
  custom_data_type: float32
  expected_columns: 8
  data_shape_on_save: channels_first
  column_names: ['EEG_Filt_1', 'EEG_Filt_2', 'EEG_Filt_3', 'EEG_Filt_4', 'EOG_Filt_1', 'EOG_Filt_2', 'EOG_Filt_3', 'EOG_Filt_4']

Successfully loaded data.
Data shape (samples, channels): (29375, 8)

First 5 rows of loaded data (transposed to samples, channels):
EEG_Filt_1 | EEG_Filt_2 | EEG_Filt_3 | EEG_Filt_4 | EOG_Filt_1 | EOG_Filt_2 | EOG_Filt_3 | EOG_Filt_4
-----------------------------------------------------------------------------------------------------
-4.332 | 28.694 | 8.617 | -2.582 | 58.043 | -2.426 | 17.328 | 1.329
-22.766 | 22.139 | 7.281 | -6.464 | 46.782 | -2.694 | 21.446 | -1.270
-55.469 | 14.363 | 5.688 | -16.487 | 31.713 | -3.230 | 28.737 | -4.439
-84.